# Greenhouse gas concentrations to emissions

Hopefully fully vectorising this will actually make things a bit easier.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d
import pickle
from climateforcing.utils import mkdir_p

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.constants.general import TIME_AXIS, GAS_BOX_AXIS
from fair21.defaults import n_gas_boxes
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
)
from fair21.gas_cycle.inverse import unstep_concentration

In [ ]:
# grab pre-computed concentrations
with open("../data/output/rcmip-fair21-concentrations.pkl","rb") as filein:
    concentration = pickle.load(filein)

In [ ]:
# # interpolate all to a monthly timestep
# t_new = np.arange(1750+1/24, 2501, 1/12)
# concentration_monthly = {}

# for specie in concentration.keys():
#     f = interp1d(np.arange(1750.5, 2501), concentration[specie], fill_value="extrapolate")
#     concentration_monthly[specie] = f(t_new)
    
# f = interp1d(np.arange(1750.5, 2501), concentration['CO2'], fill_value='extrapolate')
# concentration_co2_halfyear = f(np.arange(1750.25, 2501, 0.5))

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
# grab some emissions
emissions_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    emissions_rcmip[scenario] = {}
    for igas, gas in enumerate(gas_list):
        gas_rcmip_name = gas.replace("-", "")
        emissions_rcmip[scenario][gas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.T
    
        # CO2 and N2O units need to behave
        if gas in ('CO2', 'N2O'):
            emissions_rcmip[scenario][gas] = emissions_rcmip[scenario][gas] / 1000

In [ ]:
# grab some concentrations
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for scenario in scenarios:
    concentration_rcmip[scenario] = {}
    for gas in gas_list:
        gas_rcmip_name = gas.replace("-", "")
        concentration_rcmip[scenario][gas] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.squeeze()

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

t_new = np.arange(1750+1/24, 2501, 1/12)
f = interp1d(np.arange(1750.5, 2501), ssp245_temperature_rfmip, fill_value="extrapolate")
ssp245_temperature_monthly = f(t_new)

t_new = np.arange(1750.25, 2501, 0.5)
f = interp1d(np.arange(1750.5, 2501), ssp245_temperature_rfmip, fill_value="extrapolate")
ssp245_temperature_halfyear = f(t_new)

In [ ]:
# emissions = {}
# cumulative_emissions = {}

# for specie in concentration.keys():
#     cumulative_emissions[specie] = 0
#     gas_boxes=0
#     emissions[specie] = np.zeros(751*12)
#     airborne_emissions = 0
    
#     if specie in ("CO2", "CH4"):
#         g0, g1 = calculate_g(lifetime[specie], partition_fraction[specie])
#         for i in range(751*12): 
#             alpha_lifetime = calculate_alpha(
#                 cumulative_emissions[specie],
#                 airborne_emissions,
#                 ssp245_temperature_monthly[i],
#                 iirf_0[specie],
#                 iirf_cumulative[specie],
#                 iirf_temperature[specie],
#                 iirf_airborne[specie],
#                 g0,
#                 g1
#             )

#             emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration(
#                 concentration_monthly[specie][i],
#                 gas_boxes,
#                 airborne_emissions,
#                 burden_per_emission[specie],
#                 lifetime[specie],
#                 alpha_lifetime=alpha_lifetime,
#                 pre_industrial_concentration=pre_industrial_concentration[specie],
#                 partition_fraction=partition_fraction[specie],
#                 timestep=1/12,
#                 natural_emissions_adjustment=natural_emissions_adjustment[specie]
#             )
#             cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1/12)

#     else:
#         alpha_lifetime=1
#         for i in range(751*12): 
#             emissions[specie][i], gas_boxes, airborne_emissions = unstep_concentration(
#                 concentration_monthly[specie][i],
#                 gas_boxes,
#                 airborne_emissions,
#                 burden_per_emission[specie],
#                 lifetime[specie],
#                 alpha_lifetime=alpha_lifetime,
#                 pre_industrial_concentration=pre_industrial_concentration[specie],
#                 partition_fraction=partition_fraction[specie],
#                 timestep=1/12,
#                 natural_emissions_adjustment=natural_emissions_adjustment[specie]
#             )
#             cumulative_emissions[specie] = np.sum(emissions[specie][:i] * 1/12)
#     cumulative_emissions[specie] = np.cumsum(emissions[specie] * 1/12)

In [ ]:
# fig, ax = pl.subplots(6, 8, figsize=(16,16))
# for igas, gas in enumerate(gas_list):
#     iy = igas % 8
#     ix = igas // 8
#     ax[ix, iy].plot(np.arange(1750.5, 2501), emissions_rcmip[gas], label='RCMIP')
#     ax[ix, iy].plot(np.arange(1750+1/24, 2501, 1/12), emissions[gas], label='Back-calculated')
#     ax[ix, iy].set_title(gas)
# fig.tight_layout()

## Check with annual time stepping

In [ ]:
# turn into arrays for speed
n_gases = len(gas_list)
n_timesteps = len(concentration['ssp245']["CFC-11"])
n_scenarios = len(scenarios)

In [ ]:
concentration_array = np.ones((n_scenarios, n_gases, n_timesteps, 1)) * np.nan
for iscen, scenario in enumerate(scenarios):
    for igas, gas in enumerate(gas_list):
        gas_rcmip_name = gas.replace("-", "")
        concentration_array[iscen, igas, ...] = concentration[scenario][gas]

In [ ]:
%%time

# remember: scen, species, time, box
emissions_array = np.ones((n_scenarios, n_gases, n_timesteps, 1)) * np.nan
g0 = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
g1 = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
alpha_lifetime = np.ones((n_scenarios, n_gases, 1, 1))
airborne_emissions = np.zeros((n_scenarios, n_gases, 1, 1))
gas_boxes = np.zeros((n_scenarios, n_gases, 1, n_gas_boxes))
iirf_0_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_cumulative_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_temperature_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_airborne_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
burden_per_emission_array = np.ones((1, n_gases, 1, 1)) * np.nan
lifetime_array = np.ones((n_scenarios, n_gases, 1, n_gas_boxes)) * np.nan
pre_industrial_concentration_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
partition_fraction_array = np.zeros((n_scenarios, n_gases, 1, n_gas_boxes))
natural_emissions_adjustment_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan

cumulative_emissions_tracker = np.zeros((n_scenarios, n_gases, 1, 1))

for igas, gas in enumerate(gas_list):
    lifetime_array[:, igas, :, :] = lifetime[gas]
    partition_fraction_array[:, igas, :, :] = partition_fraction[gas]
    iirf_0_array[:, igas, :, :] = iirf_0[gas]
    iirf_cumulative_array[:, igas, :, :] = iirf_cumulative[gas]
    iirf_temperature_array[:, igas, :, :] = iirf_temperature[gas]
    iirf_airborne_array[:, igas, :, :] = iirf_airborne[gas]
    burden_per_emission_array[:, igas, :, :] = burden_per_emission[gas]
    pre_industrial_concentration_array[:, igas, :, :] = pre_industrial_concentration[gas]
    partition_fraction_array[:, igas, :, :] = partition_fraction[gas]
    natural_emissions_adjustment_array[:, igas, :, :] = natural_emissions_adjustment[gas]

g0, g1 = calculate_g(lifetime_array, partition_fraction_array)

for i_timestep in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions_tracker,
        airborne_emissions[:, :, [0], :],
        ssp245_temperature_rfmip[i_timestep],
        iirf_0_array,
        iirf_cumulative_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
    )
    alpha_lifetime[np.isnan(alpha_lifetime)]=1  # CF4 seems to have an issue. Should we raise warning?
    emissions_array[:, :, [i_timestep], :], gas_boxes, airborne_emissions = unstep_concentration(
        concentration_array[:, :, [i_timestep], :],
        gas_boxes,
        airborne_emissions,
        burden_per_emission_array,
        lifetime_array,
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration_array,
        timestep=1,
        partition_fraction=partition_fraction_array,
        natural_emissions_adjustment=natural_emissions_adjustment_array,
    )
    cumulative_emissions_tracker = cumulative_emissions_tracker + emissions_array[:, :, [i_timestep], :]
cumulative_emissions_array = np.cumsum(emissions_array, axis=TIME_AXIS)

In [ ]:
# # put back together
emissions = {}
for iscen, scenario in enumerate(scenarios):
    emissions[scenario] = {}
    for igas, gas in enumerate(gas_list):
        emissions[scenario][gas] = emissions_array[iscen, igas, :, :]

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    for iscen, scenario in enumerate(scenarios):
        ax[ix, iy].plot(np.arange(1750.5, 2501), emissions_rcmip[scenario][gas], label='RCMIP', color='r')
        ax[ix, iy].plot(np.arange(1750.5, 2501, 1), emissions[scenario][gas], label='Back-calculated', color='k')
    ax[ix, iy].set_title(gas)
fig.tight_layout()

In [ ]:
cumulative_emissions_array[0,21,:,0]